# Домашняя работа №15

Будем решать задачу сентимент-анализа. В архиве data/sentiment analisys.zip лежит несколько эксель файлов с твитами разных пользователей на разных языках (один файл - один язык).

Нужно выбрать один из файлов и провести над ним работу.

## Easy
Оценить сбалансированность классов.

Удалить пунктуацию из датасета.

Преобразовать датасет в BOW или TF-IDF, поделить на треин-тест и спрогнозировать вашей любимой моделью

In [1]:
import numpy as np 
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pymorphy2

In [2]:
df_row = pd.read_excel(r'D:\Курсы Питон\Сентимент\birulevo sentiment coded.xlsx')
df_row.head(5)

,user,tweet,SENTIMENT,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,1.0,POSITIVE
1,BorisALV,в вост. #бирюлево на одну палатку с кошатиной ...,1,NaN,2.0,NEUTRAL
2,White_technolog,у районной управы собираются местные жители #б...,2,NaN,3.0,NEGATIVE
3,White_technolog,“@tvrain: тем временем конная полиция добралас...,2,NaN,NaN,NaN
4,White_technolog,в центре появились тощие прыщавые мужичонки с ...,3,NaN,NaN,NaN


In [3]:
df = df_row[['tweet', 'SENTIMENT']].dropna()
df

,tweet,SENTIMENT
1,в вост. #бирюлево на одну палатку с кошатиной ...,1
2,у районной управы собираются местные жители #б...,2
3,“@tvrain: тем временем конная полиция добралас...,2
4,в центре появились тощие прыщавые мужичонки с ...,3
5,"как пишет товарищ в смс, на площади революции ...",3
...,...,...
1886,наш корр @dmitryhorse ведет трансляцию из #бир...,2
1887,“@gazetaru: #мид ответил на заявление германии...,2
1888,“@rt_russian: #азербайджан'ские радикалы пригр...,3
1889,“@rt_russian: эксклюзив: новые кадры задержани...,2


### Оценка сбалансированности классов

In [4]:
df.groupby(['SENTIMENT']).count()

,tweet
SENTIMENT,
1,131
2,801
3,952
IRR,6


IRR встречается всего 6 раз, этот класс исключаем из дальнейшего анализа.

In [5]:
df.drop(df[df['SENTIMENT'] == 'IRR'].index, inplace = True)
print(df)

                                                  tweet SENTIMENT
1     в вост. #бирюлево на одну палатку с кошатиной ...         1
2     у районной управы собираются местные жители #б...         2
3     “@tvrain: тем временем конная полиция добралас...         2
4     в центре появились тощие прыщавые мужичонки с ...         3
5     как пишет товарищ в смс, на площади революции ...         3
...                                                 ...       ...
1886  наш корр @dmitryhorse ведет трансляцию из #бир...         2
1887  “@gazetaru: #мид ответил на заявление германии...         2
1888  “@rt_russian: #азербайджан'ские радикалы пригр...         3
1889  “@rt_russian: эксклюзив: новые кадры задержани...         2
1890  предполагаемый убийца егора #щербаков'а в #бир...         2

[1884 rows x 2 columns]


In [6]:
dict = {1 : 'Positive', 2 : 'Neutral', 3 : 'Negative'}
df.replace({"SENTIMENT": dict}, inplace=True)
df.head(3)

,tweet,SENTIMENT
1,в вост. #бирюлево на одну палатку с кошатиной ...,Positive
2,у районной управы собираются местные жители #б...,Neutral
3,“@tvrain: тем временем конная полиция добралас...,Neutral


## Удаление пунктуации из датасета

In [7]:
import re
p = re.compile(r'[^\w\s]+')
df['tweet'] = [p.sub('', x) for x in df['tweet'].tolist()]

In [8]:
df.head(2)

,tweet,SENTIMENT
1,в вост бирюлево на одну палатку с кошатиной ст...,Positive
2,у районной управы собираются местные жители би...,Neutral


### Преобразование датасета в TF-IDF, разделение на треин-тест и прогнозирование с помощью случайного леса

In [9]:
pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("clf", RandomForestClassifier()),
    ]
)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['SENTIMENT'],
                                                    stratify=df['SENTIMENT'],
                                                    test_size=0.25)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_pred, y_test)

0.6093418259023354

## Normal
Удалить стоп-слова из датасета, использовать стеммер Портера (лежит в src/porter.py). Использовать пайплайн для TF-IDF и модели машинного обучения. Подобрать параметры. Сравнить качество нескольких моделей.

In [11]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
import nltk
stopwords = nltk.corpus.stopwords.words('russian')
print(stopwords[:10])

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Анна\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [12]:
def remove_stopwords_func(text):
    t = [token for token in text if token.lower() not in stopwords]
    text = ' '.join(t)    
    return text

In [13]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Анна\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [14]:
dff = df
dff.head(2)

,tweet,SENTIMENT
1,в вост бирюлево на одну палатку с кошатиной ст...,Positive
2,у районной управы собираются местные жители би...,Neutral


In [15]:
dff['Tweets_Tokenized'] = dff['tweet'].apply(word_tokenize)

dff.head(2)

,tweet,SENTIMENT,Tweets_Tokenized
1,в вост бирюлево на одну палатку с кошатиной ст...,Positive,"[в, вост, бирюлево, на, одну, палатку, с, коша..."
2,у районной управы собираются местные жители би...,Neutral,"[у, районной, управы, собираются, местные, жит..."


In [16]:
ps = PorterStemmer()

In [17]:
dff['Tweets_wo_Stop_Words'] = df['Tweets_Tokenized'].apply(remove_stopwords_func)

dff.head(2)

,tweet,SENTIMENT,Tweets_Tokenized,Tweets_wo_Stop_Words
1,в вост бирюлево на одну палатку с кошатиной ст...,Positive,"[в, вост, бирюлево, на, одну, палатку, с, коша...",вост бирюлево одну палатку кошатиной стало мен...
2,у районной управы собираются местные жители би...,Neutral,"[у, районной, управы, собираются, местные, жит...",районной управы собираются местные жители бирю...


In [18]:
ps.stem('lowered')

'lower'

In [19]:
ps.stem('палатку')

'палатку'

In [20]:
nltk.download('wordnet')
nltk.download('omw-1.4')

def text_pre_processing(text, flg_stemm=False, flg_lemm=True, lst_stopwords=stopwords):
    # case folding and remove punctutation
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    # tokenizing
    lst_text = text.split()
    # remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    # Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        lst_text = [ps.stem(word) for word in lst_text]
                
    # Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    #back to string from list
    text = " ".join(lst_text)
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Анна\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Анна\AppData\Roaming\nltk_data...


In [24]:
df['tweet_clean'] = df['tweet'].apply(lambda news_title: text_pre_processing(news_title))
df.head(3)

,tweet,SENTIMENT,Tweets_Tokenized,Tweets_wo_Stop_Words,tweet_clean
1,в вост бирюлево на одну палатку с кошатиной ст...,Positive,"[в, вост, бирюлево, на, одну, палатку, с, коша...",вост бирюлево одну палатку кошатиной стало мен...,вост бирюлево одну палатку кошатиной стало мен...
2,у районной управы собираются местные жители би...,Neutral,"[у, районной, управы, собираются, местные, жит...",районной управы собираются местные жители бирю...,районной управы собираются местные жители бирю...
3,tvrain тем временем конная полиция добралась д...,Neutral,"[tvrain, тем, временем, конная, полиция, добра...",tvrain временем конная полиция добралась бирюл...,tvrain временем конная полиция добралась бирюл...


In [25]:
pipeline2 = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("clf", RandomForestClassifier()),
    ]
)

In [26]:
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(df['tweet_clean'], df['SENTIMENT'],
                                                    stratify=df['SENTIMENT'],
                                                    test_size=0.25)

In [27]:
pipeline2.fit(X_train_n, y_train_n)
y_pred_rf = pipeline2.predict(X_test_n)
accuracy_score(y_pred_rf, y_test_n)

0.6326963906581741

In [29]:
from sklearn.linear_model import LogisticRegression

pipeline3 = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("lr", LogisticRegression()),
    ]
)

In [30]:
pipeline3.fit(X_train_n, y_train_n)
y_pred_lr = pipeline3.predict(X_test_n)
accuracy_score(y_pred_lr, y_test_n)

0.6751592356687898